<a href="https://colab.research.google.com/github/mefor44/NLP_disco_polo_generator/blob/main/GTP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
!pip install transformers==4.2.2
from datetime import datetime
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments,AutoModelWithLMHead,TextDataset,DataCollatorForLanguageModeling, pipeline, set_seed, TrainingArguments
drive.mount('/content/drive')

TensorFlow 1.x selected.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
model_name=   "anonymous-german-nlp/german-gpt2" #'flax-community/papuGaPT2'

model = AutoModelWithLMHead.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/675M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/971k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [3]:
train = TextDataset(tokenizer, "/content/drive/MyDrive/NLP/songs_akcent.txt", 32)
test = TextDataset(tokenizer, "/content/drive/MyDrive/NLP/songs_akcent.txt", 32)
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=TrainingArguments(output_dir="/content/drive/MyDrive/NLP/model", overwrite_output_dir=True,prediction_loss_only=True),
    data_collator=data_collator,
    train_dataset=train,
    eval_dataset=test
)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


Step,Training Loss
500,3.269700


TrainOutput(global_step=852, training_loss=2.926506651399281, metrics={'train_runtime': 304.6113, 'train_samples_per_second': 2.797, 'total_flos': 164602712162304, 'epoch': 3.0})

In [4]:
trainer.save_model()

In [5]:
from transformers import pipeline

model = pipeline('text-generation',model="/content/drive/MyDrive/NLP/model", tokenizer=model_name,config={'max_length':800})


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [6]:
model('Przez twe oczy zielone')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Przez twe oczy zielone pieśń \nW słońcu piszę pończyć śpiewam \nPowiedz jak pod śpiewam \n'}]